In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import re

In [2]:
# 1. Load and explore the dataset
df = pd.read_csv('dataset.csv')

# Remove duplicates
df = df.drop_duplicates()

In [3]:
# 2. Preprocess the data
# Convert categorical variables to numerical
categorical_cols = ['Sex', 'Hypertension', 'Diabetes', 'Level', 'Fitness Goal', 'Fitness Type']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_cats = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_cols))

In [4]:
# Scale numerical features
num_features = ['Age', 'Height', 'Weight', 'BMI']
scaler = StandardScaler()
scaled_nums = scaler.fit_transform(df[num_features])
scaled_df = pd.DataFrame(scaled_nums, columns=num_features)

# Combine processed features
X = pd.concat([scaled_df, encoded_df], axis=1)

In [5]:
# 3. Function to extract exercises
def process_exercises(exercise_str):
    exercises = exercise_str.split(',')
    return [ex.strip() for ex in exercises]

In [6]:
# 4. Function to extract diet components
def process_diet(diet_str):
    # Extract vegetables
    veg_match = re.search(r'Vegetables:\s*\((.*?)\)', diet_str)
    vegetables = veg_match.group(1).split(',') if veg_match else []
    vegetables = [v.strip() for v in vegetables]
    
    # Extract proteins
    protein_match = re.search(r'Protein Intake:\s*\((.*?)\)', diet_str)
    proteins = protein_match.group(1).split(',') if protein_match else []
    proteins = [p.strip() for p in proteins]
    
    # Extract juices
    juice_match = re.search(r'Juice:\s*\((.*?)\)', diet_str)
    juices = juice_match.group(1).split(',') if juice_match else []
    juices = [j.strip() for j in juices]
    
    return {
        'vegetables': vegetables,
        'proteins': proteins,
        'juices': juices
    }

In [7]:
# 5. Function to extract equipment
def process_equipment(equipment_str):
    equipment = equipment_str.split(',')
    return [eq.strip() for eq in equipment]

In [9]:
# 6. Recommendation System using Similarity Matching
class FitnessRecommender:
    def __init__(self, data):
        self.data = data
        self.feature_matrix = X  # Using our preprocessed features
    
    def find_similar_profiles(self, user_features, n=3):
        """Find n most similar profiles to the user's features"""
        # Convert user features to the same format as our feature matrix
        # This requires preprocessing the user input the same way we did the training data
        user_vector = self._preprocess_user_input(user_features)
        
        # Calculate similarity
        similarities = cosine_similarity(user_vector.reshape(1, -1), self.feature_matrix)
        
        # Get indices of most similar profiles
        similar_indices = np.argsort(similarities[0])[-n:][::-1]
        
        return similar_indices
    
    def _preprocess_user_input(self, user_features):
        """Preprocess user input to match the feature matrix format"""
        # Create a DataFrame with user features
        user_df = pd.DataFrame([user_features])
        
        # Extract and scale numerical features
        user_nums = scaler.transform(user_df[num_features])
        
        # One-hot encode categorical features
        user_cats = encoder.transform(user_df[categorical_cols])
        
        # Combine and return as numpy array
        user_vector = np.concatenate([user_nums.flatten(), user_cats.flatten()])
        
        return user_vector
    
    def recommend(self, user_features):
        """Generate recommendations based on user features"""
        similar_indices = self.find_similar_profiles(user_features)
        
        exercise_recs = []
        equipment_recs = []
        diet_recs = {'vegetables': [], 'proteins': [], 'juices': []}
        
        # Collect recommendations from similar profiles
        for idx in similar_indices:
            # Add exercises
            exer = process_exercises(self.data.iloc[idx]['Exercises'])
            exercise_recs.extend(exer)
            
            # Add equipment
            equip = process_equipment(self.data.iloc[idx]['Equipment'])
            equipment_recs.extend(equip)
            
            # Add diet components
            diet = process_diet(self.data.iloc[idx]['Diet'])
            diet_recs['vegetables'].extend(diet['vegetables'])
            diet_recs['proteins'].extend(diet['proteins'])
            diet_recs['juices'].extend(diet['juices'])
        
        # Remove duplicates and return recommendations
        exercise_recs = list(set(exercise_recs))
        equipment_recs = list(set(equipment_recs))
        diet_recs['vegetables'] = list(set(diet_recs['vegetables']))
        diet_recs['proteins'] = list(set(diet_recs['proteins']))
        diet_recs['juices'] = list(set(diet_recs['juices']))
        
        return {
            'exercises': exercise_recs,
            'equipment': equipment_recs,
            'diet': diet_recs
        }

In [10]:
# 7. Example usage
recommender = FitnessRecommender(df)

In [11]:
# Example user input
example_user = {
    'Sex': 'Male',
    'Age': 20,
    'Height': 1.75,
    'Weight': 60,
    'Hypertension': 'No',
    'Diabetes': 'No',
    'BMI': 19.6,  # Calculated
    'Level': 'Normal',
    'Fitness Goal': 'Weight Gain',
    'Fitness Type': 'Muscular Fitness'
}

In [12]:
# Get recommendations
recommendations = recommender.recommend(example_user)

In [14]:
# 8. Function to generate personalized recommendation text
def generate_recommendation_text(user, recommendations):
    recommendation_text = f"Personalized Fitness and Diet Plan\n\n"
    
    # Basic info
    recommendation_text += f"Profile: {user['Sex']}, {user['Age']} years old\n"
    recommendation_text += f"Height: {user['Height']}m, Weight: {user['Weight']}kg, BMI: {user['BMI']:.2f}\n"
    recommendation_text += f"Health Conditions: "
    
    if user['Hypertension'] == 'Yes':
        recommendation_text += "Hypertension, "
    if user['Diabetes'] == 'Yes':
        recommendation_text += "Diabetes, "
    if user['Hypertension'] == 'No' and user['Diabetes'] == 'No':
        recommendation_text += "None, "
    
    recommendation_text = recommendation_text.rstrip(', ') + "\n\n"
    
    # Fitness plan
    recommendation_text += "RECOMMENDED FITNESS PLAN:\n"
    recommendation_text += f"Goal: {user['Fitness Goal']}\n"
    recommendation_text += f"Type: {user['Fitness Type']}\n\n"
    
    # Exercises
    recommendation_text += "Recommended Exercises:\n"
    for i, exercise in enumerate(recommendations['exercises'], 1):
        recommendation_text += f"{i}. {exercise}\n"
    
    recommendation_text += "\nRecommended Equipment:\n"
    for i, equipment in enumerate(recommendations['equipment'], 1):
        recommendation_text += f"{i}. {equipment}\n"
    
    # Diet
    recommendation_text += "\nRECOMMENDED DIET PLAN:\n"
    
    recommendation_text += "Vegetables:\n"
    for i, veg in enumerate(recommendations['diet']['vegetables'], 1):
        recommendation_text += f"{i}. {veg}\n"
    
    recommendation_text += "\nProtein Sources:\n"
    for i, protein in enumerate(recommendations['diet']['proteins'], 1):
        recommendation_text += f"{i}. {protein}\n"
    

    
    # General advice
    recommendation_text += "\nIMPORTANT TIPS:\n"
    recommendation_text += "1. Follow a regular exercise schedule\n"
    recommendation_text += "2. Stay hydrated by drinking enough water throughout the day\n"
    recommendation_text += "3. Monitor your progress and adjust your diet and exercise routine accordingly\n"
    recommendation_text += "4. Get adequate sleep to support muscle recovery and overall health\n"
    
    if user['Hypertension'] == 'Yes' or user['Diabetes'] == 'Yes':
        recommendation_text += "5. IMPORTANT: Consult your healthcare provider before starting this program\n"
    
    return recommendation_text

In [15]:
# Generate text recommendation
personalized_recommendation = generate_recommendation_text(example_user, recommendations)
print(personalized_recommendation)

Personalized Fitness and Diet Plan

Profile: Male, 20 years old
Height: 1.75m, Weight: 60kg, BMI: 19.60
Health Conditions: None

RECOMMENDED FITNESS PLAN:
Goal: Weight Gain
Type: Muscular Fitness

Recommended Exercises:
1. bench presses
2. deadlifts
3. Squats
4. and overhead presses

Recommended Equipment:
1. Dumbbells and barbells

RECOMMENDED DIET PLAN:
Vegetables:
1. Mixed greens
2. cherry tomatoes
3. cucumbers
4. carrots
5. bell peppers
6. celery

Protein Sources:
1. tofu
2. Chicken
3. or legumes
4. fish

IMPORTANT TIPS:
1. Follow a regular exercise schedule
2. Stay hydrated by drinking enough water throughout the day
3. Monitor your progress and adjust your diet and exercise routine accordingly
4. Get adequate sleep to support muscle recovery and overall health



In [16]:
example_user = {
    'Sex': 'Female',
    'Age': 21,
    'Height': 1.65,
    'Weight': 70,
    'Hypertension': 'No',
    'Diabetes': 'Yes',
    'BMI': 20,  # Calculated
    'Level': 'Normal',
    'Fitness Goal': 'Weight Gain',
    'Fitness Type': 'Muscular Fitness'
}

In [17]:
personalized_recommendation = generate_recommendation_text(example_user, recommendations)
print(personalized_recommendation)

Personalized Fitness and Diet Plan

Profile: Female, 21 years old
Height: 1.65m, Weight: 70kg, BMI: 20.00
Health Conditions: Diabetes

RECOMMENDED FITNESS PLAN:
Goal: Weight Gain
Type: Muscular Fitness

Recommended Exercises:
1. bench presses
2. deadlifts
3. Squats
4. and overhead presses

Recommended Equipment:
1. Dumbbells and barbells

RECOMMENDED DIET PLAN:
Vegetables:
1. Mixed greens
2. cherry tomatoes
3. cucumbers
4. carrots
5. bell peppers
6. celery

Protein Sources:
1. tofu
2. Chicken
3. or legumes
4. fish

IMPORTANT TIPS:
1. Follow a regular exercise schedule
2. Stay hydrated by drinking enough water throughout the day
3. Monitor your progress and adjust your diet and exercise routine accordingly
4. Get adequate sleep to support muscle recovery and overall health
5. IMPORTANT: Consult your healthcare provider before starting this program

